# MSDS 670 Midterm Project
## Research question:


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#
# Pandas has not been playing nice with this project
# Adding options to display numbers in float format
# and make plot pretty
#
pd.options.display.float_format = '{:.0f}'.format
plt.style.use('ggplot')

In [ ]:
columns = ['country','cntry_code','series','series_code','1960','1965','1970',
           '1975','1980','1985','1990','1995','2000','2005','2010','2015','2020',
           '2022','2023','2024']
world_df = pd.read_csv('worldPopulationData/worldPopulationData.csv',skiprows=1,
                        names = columns)
world_df.columns

In [ ]:
stop

In [ ]:
world_df.drop(['cntry_code', 'series_code'], inplace=True, axis=1)

In [ ]:
world_df.info()

In [ ]:
world_df.head()

In [ ]:
countries = ['United States','China','Germany','Japan','India',
             'United Kingdom','France','Italy','Brazil','Canada']

In [ ]:
us = world_df[world_df['country'] == 'United States']
china = world_df[world_df['country'] == 'China']
germany = world_df[world_df['country'] == 'Germany']
japan = world_df[world_df['country'] == 'Japan']
india = world_df[world_df['country'] 

In [ ]:
brazil = world_df[world_df['country'] == 'Brazil']
drop_cols = ['country','series','2022','2023','2024']
br_birth = brazil.iloc[0]
br_birth.drop(drop_cols, inplace=True)
br_death = brazil.iloc[1]
br_death.drop(drop_cols, inplace=True)
plt.figure(figsize=(10,6))
plt.title('Births and Deaths Per 1000 (Brazil)')
plt.xlabel('Year')
plt.ylabel('Per 1000')
plt.plot(br_birth, color='darkorange', label='Births', marker='o')
plt.plot(br_death, color='black', label='Deaths', marker='o')
plt.legend()
plt.show()

In [ ]:
canada = world_df[world_df['country'] == 'Canada']
drop_cols = ['country','series','2022','2023','2024']
ca_birth = canada.iloc[0]
ca_birth.drop(drop_cols, inplace=True)
ca_death = canada.iloc[1]
ca_death.drop(drop_cols, inplace=True)
plt.figure(figsize=(10,6))
plt.title('Births and Deaths Per 1000 (Candada)')
plt.xlabel('Year')
plt.ylabel('Per 1000')
plt.plot(ca_birth, color='darkorange', label='Births', marker='o')
plt.plot(ca_death, color='black', label='Deaths', marker='o')
plt.legend()
plt.show()

In [ ]:
china = world_df[world_df['country'] == 'China']
drop_cols = ['country','series','2022','2023','2024']
cn_birth = china.iloc[0]
cn_birth.drop(drop_cols, inplace=True)
cn_death = china.iloc[1]
cn_death.drop(drop_cols, inplace=True)
plt.figure(figsize=(10,6))
plt.title('Births and Deaths Per 1000 (China)')
plt.xlabel('Year')
plt.ylabel('Per 1000')
plt.plot(cn_birth, color='darkorange', label='Births', marker='o')
plt.plot(cn_death, color='black', label='Deaths', marker='o')
plt.legend()
plt.show()

In [ ]:
france = world_df[world_df['country'] == 'France']
drop_cols = ['country','series','2022','2023','2024']
fr_birth = france.iloc[0]
fr_birth.drop(drop_cols, inplace=True)
fr_death = france.iloc[1]
fr_death.drop(drop_cols, inplace=True)
plt.figure(figsize=(10,6))
plt.title('Births and Deaths Per 1000 (France)')
plt.xlabel('Year')
plt.ylabel('Per 1000')
plt.plot(fr_birth, color='blue', label='Births', marker='o')
plt.plot(fr_death, color='black', label='Deaths', marker='o')
plt.legend()
plt.show()

In [ ]:
germany = world_df[world_df['country'] == 'Germany']
drop_cols = ['country','series','2022','2023','2024']
de_birth = germany.iloc[0]
de_birth.drop(drop_cols, inplace=True)
de_death = germany.iloc[1]
de_death.drop(drop_cols, inplace=True)
plt.figure(figsize=(10,6))
plt.title('Births and Deaths Per 1000 (Germany)')
plt.xlabel('Year')
plt.ylabel('Per 1000')
plt.plot(de_birth, color='darkorange', label='Births', marker='o')
plt.plot(de_death, color='black', label='Deaths', marker='o')
plt.legend()
plt.show()

In [ ]:
india = world_df[world_df['country'] == 'India']
drop_cols = ['country','series','2022','2023','2024']
in_birth = india.iloc[0]
in_birth.drop(drop_cols, inplace=True)
in_death = india.iloc[1]
in_death.drop(drop_cols, inplace=True)
plt.figure(figsize=(10,6))
plt.title('Births and Deaths Per 1000 (India)')
plt.xlabel('Year')
plt.ylabel('Per 1000')
plt.plot(in_birth, color='darkorange', label='Births', marker='o')
plt.plot(in_death, color='black', label='Deaths', marker='o')
plt.legend()
plt.show()

In [ ]:
italy = world_df[world_df['country'] == 'Italy']
drop_cols = ['country','series','2022','2023','2024']
it_birth = italy.iloc[0]
it_birth.drop(drop_cols, inplace=True)
it_death = italy.iloc[1]
it_death.drop(drop_cols, inplace=True)
plt.figure(figsize=(10,6))
plt.title('Births and Deaths Per 1000 (Italy)')
plt.xlabel('Year')
plt.ylabel('Per 1000')
plt.plot(it_birth, color='darkorange', label='Births', marker='o')
plt.plot(it_death, color='black', label='Deaths', marker='o')
plt.legend()
plt.show()

In [ ]:
japan = world_df[world_df['country'] == 'Japan']
jp_total = japan.iloc[2]
drop_cols = ['country','series','2022','2023','2024']
jp_birth = japan.iloc[0]
jp_birth.drop(drop_cols, inplace=True)
jp_death = japan.iloc[1]
jp_death.drop(drop_cols, inplace=True)
plt.figure(figsize=(10,6))
plt.title('Births and Deaths Per 1000 (Japan)')
plt.xlabel('Year')
plt.ylabel('Per 1000')
plt.plot(jp_birth, color='darkorange', label='Births', marker='o')
plt.plot(jp_death, color='black', label='Deaths', marker='o')
plt.legend()
plt.show()

In [ ]:
uk = world_df[world_df['country'] == 'United Kingdom']
drop_cols = ['country','series','2022','2023','2024']
uk_birth = uk.iloc[0]
uk_birth.drop(drop_cols, inplace=True)
uk_death = uk.iloc[1]
uk_death.drop(drop_cols, inplace=True)
plt.figure(figsize=(10,6))
plt.title('Births and Deaths Per 1000 (United Kingdom)')
plt.xlabel('Year')
plt.ylabel('Per 1000')
plt.plot(uk_birth, color='darkorange', label='Births', marker='o')
plt.plot(uk_death, color='black', label='Deaths', marker='o')
plt.legend()
plt.show()

In [ ]:
us = world_df[world_df['country'] == 'United States']
drop_cols = ['country','series','2022','2023','2024']
us_birth = us.iloc[0]
us_birth.drop(drop_cols, inplace=True)
us_death = us.iloc[1]
us_death.drop(drop_cols, inplace=True)
plt.figure(figsize=(10,6))
plt.title('Births and Deaths Per 1000 (United States)')
plt.xlabel('Year')
plt.ylabel('Per 1000')
plt.plot(us_birth, color='darkorange', label='Births', marker='o')
plt.plot(us_death, color='black', label='Deaths', marker='o')
plt.legend()
plt.show()

In [ ]:
br_total = brazil.iloc[2]
br_total.drop(['country', 'series'], inplace=True)
br_total = br_total.astype('int64')
br_scaled = br_total.apply(lambda x: x/1000000)
plt.figure(figsize=(10,6))
plt.title('Population Total (Brazil)')
plt.xlabel('Year')
plt.ylabel('Population in Millions')
plt.plot(br_scaled, color='black', marker='o')
plt.show()

In [ ]:
ca_total = canada.iloc[2]
ca_total.drop(['country', 'series'], inplace=True)
ca_total = ca_total.astype('int64')
ca_scaled = ca_total.apply(lambda x: x/1000000)
plt.figure(figsize=(10,6))
plt.title('Population Total (Canada)')
plt.xlabel('Year')
plt.ylabel('Population in Millions')
plt.plot(ca_scaled, color='black', marker='o')
plt.show()

In [ ]:
cn_total = china.iloc[2]
cn_total.drop(['country', 'series'], inplace=True)
cn_total = cn_total.astype('int64')
cn_scaled = cn_total.apply(lambda x: x/1000000)
plt.figure(figsize=(10,6))
plt.title('Population Total (China)')
plt.xlabel('Year')
plt.ylabel('Population in Millions')
plt.plot(cn_scaled, color='black', marker='o')
plt.show()

In [ ]:
fr_total = france.iloc[2]
fr_total.drop(['country', 'series'], inplace=True)
fr_total = fr_total.astype('int64')
fr_scaled = fr_total.apply(lambda x: x/1000000)
plt.figure(figsize=(10,6))
plt.title('Population Total (France)')
plt.xlabel('Year')
plt.ylabel('Population in Millions')
plt.plot(fr_scaled, color='black', marker='o')
plt.show()

In [ ]:
de_total = germany.iloc[2]
de_total.drop(['country', 'series'], inplace=True)
de_total = de_total.astype('int64')
de_scaled = de_total.apply(lambda x: x/1000000)
plt.figure(figsize=(10,6))
plt.title('Population Total (Germany)')
plt.xlabel('Year')
plt.ylabel('Population in Millions')
plt.plot(de_scaled, color='black', marker='o')
plt.show()

In [ ]:
in_total = india.iloc[2]
in_total.drop(['country', 'series'], inplace=True)
in_total = in_total.astype('int64')
in_scaled = in_total.apply(lambda x: x/1000000)
plt.figure(figsize=(10,6))
plt.title('Population Total (India)')
plt.xlabel('Year')
plt.ylabel('Population in Millions')
plt.plot(in_scaled, color='black', marker='o')
plt.show()

In [ ]:
it_total = italy.iloc[2]
it_total.drop(['country', 'series'], inplace=True)
it_total = it_total.astype('int64')
it_scaled = it_total.apply(lambda x: x/1000000)
plt.figure(figsize=(10,6))
plt.title('Population Total (Italy)')
plt.xlabel('Year')
plt.ylabel('Population in Millions')
plt.plot(it_scaled, color='black', marker='o')
plt.show()

In [ ]:
jp_total = japan.iloc[2]
jp_total.drop(['country', 'series'], inplace=True)
jp_total = jp_total.astype('int64')
jp_scaled = jp_total.apply(lambda x: x/1000000)
plt.figure(figsize=(10,6))
plt.title('Population Total (Japan)')
plt.xlabel('Year')
plt.ylabel('Population in Millions')
plt.plot(jp_scaled, color='black', marker='o')
plt.show()

In [ ]:
uk_total = uk.iloc[2]
uk_total.drop(['country', 'series'], inplace=True)
uk_total = uk_total.astype('int64')
uk_scaled = uk_total.apply(lambda x: x/1000000)
plt.figure(figsize=(10,6))
plt.title('Population Total (United Kingdom)')
plt.xlabel('Year')
plt.ylabel('Population in Millions')
plt.plot(uk_scaled, color='black', marker='o')
plt.show()

In [ ]:
us_total = us.iloc[2]
us_total.drop(['country', 'series'], inplace=True)
us_total = us_total.astype('int64')
us_scaled = us_total.apply(lambda x: x/1000000)
plt.figure(figsize=(10,6))
plt.title('Population Total (United States)')
plt.xlabel('Year')
plt.ylabel('Population in Millions')
plt.plot(us_scaled, color='black', marker='o')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.title('Population Total Top Ten Economies')
plt.xlabel('Year')
plt.ylabel('Population in Millions')
plt.plot(br_scaled, color='blue', label='Brazil', marker='o')
plt.plot(ca_scaled, color='orange', label='Canada', marker='o')
plt.plot(cn_scaled, color='purple', label='China', marker='o')
plt.plot(fr_scaled, color='brown', label='France', marker='o')
plt.plot(de_scaled, color='olive', label='Germany', marker='o')
plt.plot(in_scaled, color='cyan', label='India', marker='o')
plt.plot(it_scaled, color='navy', label='Italy', marker='o')
plt.plot(jp_scaled, color='teal', label='Japan', marker='o')
plt.plot(uk_scaled, color='yellow', label='United Kingdom', marker='o')
plt.plot(us_scaled, color='black', label='United States', marker='o')
plt.legend()
plt.show()